# Get Nandos locations

#### Load Python tools and Jupyter config

In [1]:
import us 
import json
import black
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
place = "nandos"
place_formal = "Nando's"
color = "#e6007d"

## Read data

#### All the locations

In [4]:
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

In [5]:
response = requests.get("https://www.nandosperiperi.com/find/", headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
json_data = json.loads(soup.find("script", id="__NEXT_DATA__").string)

In [7]:
src = pd.DataFrame(json_data["props"]["pageProps"]["allLocations"]["restaurants"])

In [8]:
df = (
    src[
        [
            "id",
            "name",
            "streetAddress",
            "city",
            "state",
            "zipCode",
            "latitude",
            "longitude",
            "telephone",
        ]
    ]
    .rename(columns={"streetAddress": "street", "telephone": "phone"})
    .copy()
)

#### The result? 

In [9]:
df.head()

,id,name,street,city,state,zipCode,latitude,longitude,phone
0,98,Peachtree Corners,"5161 Peachtree PKWY, #605",Norcross,GA,30092,33.98000647063222,-84.21784185109394,None
1,99,PERimeter,"120 High Street, Suite 210",Dunwoody,GA,30346,33.924941,-84.346361,None
2,TX,Mueller,1825 McBee Street,Austin,TX,78723,30.297298,-97.702375,None
3,127951,Waugh Chapel,1403 S. Main Chapel Way Ste 111,Gambrills,MD,21054,39.0322832,-76.6856883,14433026225
4,127952,Baltimore St,421 W. Baltimore St,Baltimore,MD,21201,39.2890885,-76.621934,14436813675


#### How many locations?

In [10]:
len(df)

48

#### Create a mapping of state abbreviations to full state names using the us library

In [11]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [12]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [13]:
df["brand"] = place_formal

---

## Geography

#### Make it a geodataframe

In [14]:
df_geo = df.copy()

In [15]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

---

## Maps

#### US states background

In [16]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [17]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [18]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [19]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [20]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [21]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)